# Adversarial example using pytorch

In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use pytorch for training the neural network.

We don't detail the optimization model here. Please refer to the example in the documentation.

This example requires the additional packages:
 - [torch](https://pytorch.org/)
 - [matplotlib](https://matplotlib.org/)
 - [skorch](https://skorch.readthedocs.io/en/stable/)

 The latter package is a wrapper for giving to pytorch a Scikit-Learn interface.

## Import the necessary packages and load data

We import all the package we need for this example.
We fetch the MINST data set using sklearn's functionalities.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_openml

import torch
from skorch import NeuralNetClassifier

import gurobipy as gp
from gurobi_ml import add_predictor_constr

In [ ]:
# Get MNIST digit recognition data set
mnist = fetch_openml("mnist_784")
X, y = mnist.data, mnist.target

The data imported is in pandas data frames. We don't really have a use for those here and working with numpy is simpler for pytorch so convert them to numpy. For pytorch, we also have to convert them to the appropriate type. Finally, scale the input.

In [ ]:
X = X.to_numpy()
y = y.to_numpy()
X = X.astype(np.float32)
y = y.astype(np.int64)

X /= 255.0  # scaling

## Construct and train the neural network

We construct a sequential neural network with 2 hidden layers of 50 neurons.
To train it, we use `skorch` that provides an interface similar to `scikit-learn`.

In [ ]:
nn_model = torch.nn.Sequential(
    torch.nn.Linear(28*28, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 10),
    torch.nn.Softmax(1)
)

In [ ]:
clf = NeuralNetClassifier(nn_model, max_epochs=50,
                          lr=0.1, iterator_train__shuffle=True,)
trainsize = 60000
clf.fit(X=X[:trainsize], y=y[:trainsize])

In [ ]:
print(f"Training score: {clf.score(X[:trainsize], y[:trainsize])}")
print(f"Validation set score: {clf.score(X[trainsize:], y[trainsize:])}")

In [ ]:
nn_regression = nn.Sequential(*[layer for layer in nn_model[:-1]])

In [ ]:
first_image = X[10000, :]
pixels = first_image.reshape((28, 28))
plt.imshow(pixels, cmap="gray")
plt.show()

In [ ]:
nn_regression.forward(torch.from_numpy(first_image))

In [ ]:
m = gp.Model()
x = m.addMVar(first_image.shape, lb=0.0, ub=1.0)
y = m.addMVar((10), lb=-gp.GRB.INFINITY)
abs_diff = m.addMVar(first_image.shape, lb=0.0, ub=1.0)


In [ ]:
add_predictor_constr(m, nn_regression, x, y)

In [ ]:
m.addConstr(abs_diff >= x - first_image)
m.addConstr(abs_diff >= first_image - x)
m.addConstr(abs_diff.sum() <= 5)

In [ ]:
m.setObjective(y[5] - y[3], gp.GRB.MAXIMIZE)

In [ ]:
m.optimize()